In [65]:
import os
import pandas as pd
import csv
# Path to the folder containing files
folder_path = parent_folder_path = data_folder = './Data/'

# Function to read data from catalog_id file
def read_catalog_file(catalog_file):
    df = pd.read_csv(catalog_file, header=None)
    # Extract values from 2nd to 5th row
    values = df.iloc[1:5, 0].tolist()
    return values

# Function to update corresponding table_id file
def update_table_file(table_file, values):
    df = pd.read_csv(table_file)
    # Delete the last row
    df.drop(df.tail(1).index, inplace=True)
    # Insert values as new columns at the beginning of the DataFrame after date,and period
    df.insert(2, 'State', values[3])
    df.insert(2, 'State_or_MSA', values[2])
    df.insert(2, 'Area', values[1])
    df.insert(2, 'Seasonally Adjusted', values[0])
    df.to_csv(table_file, index=False, mode='w')  # Overwrite existing file
    
def get_column_names(csv_file):
    df = pd.read_csv(csv_file)
    return list(df.columns)

# Function to concatenate CSV files with matching columns
def concatenate_matching_files(matching_files, folder_path, name):
    dfs = []
    for filename in matching_files:
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_file_path = os.path.join(folder_path, f'{name}.csv')
    concatenated_df.to_csv(concatenated_file_path, index=False, mode='w')

# Function to add extra column from one file to another for CSV files with non-matching columns
def add_extra_column(non_matching_files, folder_path):
    file1_path = os.path.join(folder_path, non_matching_files[0])
    file2_path = os.path.join(folder_path, non_matching_files[1])

    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)

    extra_column = set(df1.columns) - set(df2.columns)
    if extra_column:
        extra_column = extra_column.pop()
        df2.insert(df1.columns.get_loc(extra_column), extra_column, '')
        df2.to_csv(file2_path, index=False, mode='w')
    




In [72]:
# Iterate through each folder in the directory, Add data from catalog and delete catalog files
for foldername in os.listdir(folder_path):
    folder_fullpath = os.path.join(folder_path, foldername)
    if os.path.isdir(folder_fullpath):
        # Iterate through each file in the folder
        for filename in os.listdir(folder_fullpath):
            if filename.startswith('catalog'):
                # Extract id from filename
                catalog_id = filename[7:-4]
                # Read values from catalog_id file
                values = read_catalog_file(os.path.join(folder_fullpath, filename))
                # Find corresponding table_id file
                table_filename = f'table{catalog_id}.csv'
                table_file_path = os.path.join(folder_fullpath, table_filename)
                if os.path.exists(table_file_path):
                    # Update corresponding table_id file
                    update_table_file(table_file_path, values)
                    # Delete the catalog file
                    os.remove(os.path.join(folder_fullpath, filename))
                    
# Iterate through each folder in the parent folder
for folder_name in os.listdir(parent_folder_path):
    folder_path = os.path.join(parent_folder_path, folder_name)
    matching_files = []
    non_matching_files = []

    # Iterate through each file in the folder
    for filename1 in os.listdir(folder_path):
        if filename1.endswith('.csv'):
            file_path1 = os.path.join(folder_path, filename1)
            column_names1 = get_column_names(file_path1)

            # Check if the column "State_or_MSA" exists and its value is "Statewide"
            if 'State_or_MSA' in column_names1:
                with open(file_path1, 'r') as file:
                    reader = csv.DictReader(file)
                    for row in reader:
                        if row['State_or_MSA'] == 'Statewide':
                            non_matching_files.append(filename1)
                            break
                    else:
                        matching_files.append(filename1)  # This line moved inside the else block
            else:
                non_matching_files.append(filename1)

#     print("matching_files", matching_files)
#     print('non_matching_files', non_matching_files)

    # Concatenate matching files
    if len(matching_files) > 1:
        concatenate_matching_files(matching_files, folder_path, 'MSA')

    # Add extra column for non-matching files
    if len(non_matching_files) == 2:
        add_extra_column(non_matching_files, folder_path)
        concatenate_matching_files(non_matching_files, folder_path, 'State')


In [73]:
# Initialize empty lists to store dataframes
msa_dfs = []
state_dfs = []

# Iterate through each folder in the Data directory
for folder in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder)
    if os.path.isdir(folder_path):
        # Check if MSA.csv and state.csv files exist in the folder
        msa_file = os.path.join(folder_path, 'MSA.csv')
        state_file = os.path.join(folder_path, 'State.csv')
        if os.path.exists(msa_file):
            msa_dfs.append(pd.read_csv(msa_file))
        if os.path.exists(state_file):
            state_dfs.append(pd.read_csv(state_file))

# Concatenate all MSA dataframes and save to a single CSV file
msa_concatenated = pd.concat(msa_dfs)
msa_concatenated.to_csv('concatenated_msa.csv', index=False, mode='w')

# Concatenate all state dataframes and save to a single CSV file
state_concatenated = pd.concat(state_dfs)
state_concatenated.to_csv('concatenated_state.csv', index=False, mode='w')


In [75]:
state_concatenated

,Year,Period,Seasonally Adjusted,Area,State_or_MSA,State,civilian noninstitutional population,labor force participation rate,employment-population ratio,labor force,employment,unemployment,unemployment rate
0,2014,Jan,Not Seasonally Adjusted,Wyoming,Statewide,Wyoming,449367,66.8,63.3,300163,284285,15878,5.3
1,2014,Feb,Not Seasonally Adjusted,Wyoming,Statewide,Wyoming,449336,66.8,63.4,299953,284800,15153,5.1
2,2014,Mar,Not Seasonally Adjusted,Wyoming,Statewide,Wyoming,449330,67.1,63.8,301417,286828,14589,4.8
3,2014,Apr,Not Seasonally Adjusted,Wyoming,Statewide,Wyoming,449340,66.7,63.9,299933,287064,12869,4.3
4,2014,May,Not Seasonally Adjusted,Wyoming,Statewide,Wyoming,449389,67.2,64.3,302018,289087,12931,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,2023,Sep,Seasonally Adjusted,Puerto Rico,Statewide,Puerto Rico,NaN,NaN,NaN,1209136(R),1140616(R),68520(R),5.7(R)
238,2023,Oct,Seasonally Adjusted,Puerto Rico,Statewide,Puerto Rico,NaN,NaN,NaN,1215150(R),1147078(R),68072(R),5.6(R)
239,2023,Nov,Seasonally Adjusted,Puerto Rico,Statewide,Puerto Rico,NaN,NaN,NaN,1214428(R),1145996(R),68432(R),5.6(R)
240,2023,Dec,Seasonally Adjusted,Puerto Rico,Statewide,Puerto Rico,NaN,NaN,NaN,1212497(R),1143724(R),68773(R),5.7(R)


In [110]:
from datetime import datetime
from calendar import monthrange

# Load CSV files
state_df = pd.read_csv('concatenated_state.csv')
msa_df = pd.read_csv('concatenated_msa.csv')




In [112]:
msa_df

,Year,Period,Seasonally Adjusted,Area,State_or_MSA,State,labor force,employment,unemployment,unemployment rate,civilian noninstitutional population,labor force participation rate,employment-population ratio
0,2014,Jan,Not Seasonally Adjusted,"Blacksburg-Christiansburg-Radford, VA Metropol...",Metropolitan areas,Virginia,89092,83884,5208,5.8,NaN,NaN,NaN
1,2014,Feb,Not Seasonally Adjusted,"Blacksburg-Christiansburg-Radford, VA Metropol...",Metropolitan areas,Virginia,93408,88128,5280,5.7,NaN,NaN,NaN
2,2014,Mar,Not Seasonally Adjusted,"Blacksburg-Christiansburg-Radford, VA Metropol...",Metropolitan areas,Virginia,94054,88648,5406,5.7,NaN,NaN,NaN
3,2014,Apr,Not Seasonally Adjusted,"Blacksburg-Christiansburg-Radford, VA Metropol...",Metropolitan areas,Virginia,92852,88466,4386,4.7,NaN,NaN,NaN
4,2014,May,Not Seasonally Adjusted,"Blacksburg-Christiansburg-Radford, VA Metropol...",Metropolitan areas,Virginia,91731,86564,5167,5.6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45612,2023,Sep,Not Seasonally Adjusted,"San Juan-Carolina-Caguas, PR Metropolitan Stat...",Metropolitan areas,Puerto Rico,806875,768356,38519,4.8,NaN,NaN,NaN
45613,2023,Oct,Not Seasonally Adjusted,"San Juan-Carolina-Caguas, PR Metropolitan Stat...",Metropolitan areas,Puerto Rico,817051,778379,38672,4.7,NaN,NaN,NaN
45614,2023,Nov,Not Seasonally Adjusted,"San Juan-Carolina-Caguas, PR Metropolitan Stat...",Metropolitan areas,Puerto Rico,822539,782253,40286,4.9,NaN,NaN,NaN
45615,2023,Dec,Not Seasonally Adjusted,"San Juan-Carolina-Caguas, PR Metropolitan Stat...",Metropolitan areas,Puerto Rico,814250(P),774076(P),40174(P),4.9(P),NaN,NaN,NaN


In [ ]:
# Define a function to convert month name to its corresponding number
def month_to_num(month_name):
    return datetime.strptime(month_name, '%b').month

# Define a function to calculate the last day of the month
def last_day_of_month(year, month):
    return monthrange(year, month)[1]

# Convert Period column to month number
state_df['Month'] = state_df['Period'].apply(lambda x: month_to_num(x))
msa_df['Month'] = msa_df['Period'].apply(lambda x: month_to_num(x))

# Create Date column
state_df['Date'] = state_df.apply(lambda row: f"{last_day_of_month(row['Year'], row['Month']):02d}/{row['Month']:02d}/{row['Year']}", axis=1)
msa_df['Date'] = msa_df.apply(lambda row: f"{last_day_of_month(row['Year'], row['Month']):02d}/{row['Month']:02d}/{row['Year']}", axis=1)

# Drop the original Year and Period columns
state_df.drop(columns=['Year', 'Period', 'Month'], inplace=True)
msa_df.drop(columns=['Year', 'Period', 'Month'], inplace=True)

# Rearrange columns to have 'Date' column as the first column
state_df = state_df[['Date'] + [col for col in state_df.columns if col != 'Date']]
msa_df = msa_df[['Date'] + [col for col in msa_df.columns if col != 'Date']]

# Save the modified dataframes to CSV files if needed
state_df.to_csv('concatenated_state_modified.csv', index=False, mode='w')
msa_df.to_csv('concatenated_msa_modified.csv', index=False, mode='w')